# Water Column Modelling for USGS

In [1]:
# Import water column model class and components
from water_column_model.column import Column
from water_column_model.advance import *
from water_column_model.params import *
from water_column_model.params import A,B,C,E
# Data Science and Visualization Imports
import pandas as pd
import xarray as xr
import numpy as np
# Miscellaneous
from tqdm import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Importing USGS Data

In [2]:
filename = 'data/USGS_veg_density.csv'
wcolumns = pd.read_csv(filename)

Setting Water Column Heights

In [3]:
#  (arbitrary for now, but based on average tidal height)
H = 2.8072 # meters

# Adjusting Tide level to topography (elevation) for correct water column height for each site
wcolumns['wc_height'] = H - wcolumns['Elevation'] # meters

# Pulling parameters
params = [A, B, C, Sq, kappa, nu, g, rho0, alpha]

#wcolumns = wcolumns.sample(5)

Dropping out unstable cases

In [4]:
wcolumns['dz'] = wcolumns['wc_height']/80
nu = 1e-6
dt = 60
wcolumns['stable'] = (wcolumns['dz'])**2 / nu >= dt
# Only keeping stable cases
wcolumns = wcolumns.loc[wcolumns['stable'] == True]
#wcolumns = wcolumns.sample(5)
wcolumns

,Season,date,site,Elevation,channel_dist,ave_ht,cover,vol_est,vol_calc,density_naive,density_final,frontal_total,wc_height,dz,stable
0,Summer,7/28/2021,Bay,2.1313,0.5,0.065,0.10,6500,233.351057,267.040006,630.481094,1103.449997,0.6759,0.008449,True
5,Summer,7/28/2021,Bay,2.1789,48.0,0.160,0.75,120000,4308.019514,4929.969345,4728.608206,20371.384570,0.6283,0.007854,True
6,Summer,7/28/2021,Bay,2.1308,72.0,0.240,0.65,156000,5600.425368,6408.960148,4098.127112,26482.799940,0.6764,0.008455,True
7,Summer,7/28/2021,Bay,2.1450,0.5,0.060,0.25,15000,538.502439,616.246168,1576.202735,2546.423071,0.6622,0.008277,True
12,Summer,7/28/2021,Bay,2.1816,48.0,0.110,0.85,93500,3356.665204,3841.267781,5359.089301,15872.703810,0.6256,0.007820,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Winter,1/25/2022,Interior,2.1294,2.0,0.130,0.75,97500,3500.265855,4005.600093,4728.608206,16551.749960,0.6778,0.008472,True
110,Winter,1/25/2022,Interior,2.1310,6.0,0.190,0.80,152000,5456.824717,6244.627837,5043.848753,25803.753780,0.6762,0.008453,True
111,Winter,1/25/2022,Interior,2.1040,12.0,0.140,0.65,91000,3266.914798,3738.560086,4098.127112,15448.299960,0.7032,0.008790,True
112,Winter,1/25/2022,Interior,2.0919,24.0,0.200,0.85,170000,6103.027644,6984.123238,5359.089301,28859.461470,0.7153,0.008941,True


Column Initialization and Time Stepping Function

In [5]:
# for a given set of parameteres, create a column, run the model and return Flow Properties
def run_column_model(N, H, L, SMALL, params, veg_density, veg_ht, density_alpha):
    # Initializing Column Object
    col = Column(N, H, L, SMALL)
    A = params[0]
    B = params[1]
    C = params[2]
    Sq = params[3]
    kappa = params[4]
    nu = params[5]
    g = params[6]
    rho0 = params[7]
    alpha = params[8]
    # Setting up vlaues and importing Vegetation Distribution from Data
    col = col.setup(A, B, C, Sq, kappa, SMALL, nu, g, rho0, alpha)
    col = col.import_veg(density_alpha, veg_density, veg_ht)
    
    # Running Model for Col.M amount of time (See Column Class to change M)
    t = [] 
    for i in tqdm(range(col.M), leave=False):
        t.append(col.dt*(i))
        # Unew, Cnew, Qnew, Q2new, Q2Lnew, rhonew, Lnew, nu_tnew, Kznew, Kqnew, N_BVnew, N_BVsqnew
        [col.U, col.scalar, col.Q, col.Q2, col.Q2L, col.rho, col.L, col.nu_t, col.Kz, col.Kq, col.N_BV, col.N_BVsq] = wc_advance(col, t_px, px0, t[i])
        # Have TQDM instead print('Step! t=' +str(t[i])+'s')
    ### Only for now ### Return Velocity Profile ### Will implement returning more later
    col_res = pd.DataFrame({'U Velocity':col.U,
                            'Q':col.Q,
                            'Q2':col.Q2,
                            'Q2L':col.Q2L,
                            'Z':col.z,
                            'Kq':col.Kq,
                            'nu_t':col.nu_t,
                            })
    return col_res

Model Runs Function

In [6]:
def run_collection(n_runs, alpha_list):
    
    export_baseline = 'model_runs/baseline_wc_full.nc'
    export_runs = 'model_runs/'
    
    # Making new dataframes for specific sites
    wcol_bay = wcolumns[wcolumns['site'] == 'Bay']
    wcol_int = wcolumns[wcolumns['site'] == 'Interior']
    wcol_creek = wcolumns[wcolumns['site'] == 'Creek']

    # Running and exporting water column model for base case
    col_res_base = wcolumns.apply(lambda x: run_column_model(80,(x.wc_height),0,SMALL,params,x.density_final,(x.ave_ht),density_alpha=0), axis=1)
    ds_base = xr.concat([df.to_xarray() for df in col_res_base], dim="Model Runs")
    ds_base = ds_base.assign_attrs(description='Base Model run results for all sites.')    
    ds_base.to_netcdf(path=export_baseline, mode='w')

    # Running and exporting water column model for vegetated cases
    for i in tqdm(range(n_runs), leave=False):
        
        alpha_ask = alpha_list[i]
        # Initializing and running water column model for each entry
        col_res_all = wcol_bay.apply(lambda x: run_column_model(80,(x.wc_height),0,SMALL,params,x.density_final,(x.ave_ht),alpha_ask), axis=1)
        col_res_bay = wcol_bay.apply(lambda x: run_column_model(80,(x.wc_height),0,SMALL,params,x.density_final,(x.ave_ht),alpha_ask), axis=1)
        col_res_creek = wcol_creek.apply(lambda x: run_column_model(80,(x.wc_height),0,SMALL,params,x.density_final,(x.ave_ht),alpha_ask), axis=1)
        col_res_interior = wcol_int.apply(lambda x: run_column_model(80,(x.wc_height),0,SMALL,params,x.density_final,(x.ave_ht),alpha_ask), axis=1)
  
        # Turning results into xArray for easy visualization and manip
        ds_all = xr.concat([df.to_xarray() for df in col_res_all], dim="Model Runs")
        ds_all = ds_all.assign_attrs(description='Model run results for all sites.')

        ds_bay = xr.concat([df.to_xarray() for df in col_res_bay], dim="Model Runs")
        ds_bay = ds_bay.assign_attrs(description='Model run results for Bay site.')

        ds_creek = xr.concat([df.to_xarray() for df in col_res_creek], dim="Model Runs")
        ds_creek = ds_creek.assign_attrs(description='Model run results for Creek site.')

        ds_int = xr.concat([df.to_xarray() for df in col_res_interior], dim="Model Runs")
        ds_int = ds_int.assign_attrs(description='Model run results for Interior site.')

        # Export model runs    
        ds_all.to_netcdf(path=export_runs+'_wc_all_'+str(i)+'.nc', mode='w')
        ds_bay.to_netcdf(path=export_runs+'_wc_bay_'+str(i)+'.nc', mode='w')
        ds_creek.to_netcdf(path=export_runs+'_wc_creek_'+str(i)+'.nc', mode='w')
        ds_int.to_netcdf(path=export_runs+'_wc_interior_'+str(i)+'.nc', mode='w')

In [7]:
wcol_bay = wcolumns[wcolumns['site'] == 'Bay']
col_res_all = run_column_model(80,(wcol_bay.iloc[0].wc_height),0,SMALL,params,wcol_bay.iloc[0].density_final,(wcol_bay.iloc[0].ave_ht),0.1)

In [8]:
# number of elements
n = int(input("Enter number of model runs : "))
# Different alpha will be asked for every new run
alpha_list = list(map(float,input("\nInput " + str(n) + " LIST of values [0-1] for alpha density coefficient: : ").strip().split()))[:n]

run_collection(n, alpha_list)